In [1]:
#The text in the document by <Bhavana_Shanmugam> is licensed under CC BY 3.0 https://creativecommons.org/licenses/by/3.0/us/
#The code in the document by <Bhavana_Shanmugam> is licensed under the MIT License https://opensource.org/licenses/MIT

import urllib
import urllib.request as uReq
import bs4
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep
from time import time
from random import randint
import pickle
import requests
from requests import get

#Creating lists
names = []
imdb_ratings = []
metascores = []
genre = []
runtime = []

start_time = time()
requests = 0

headers = {"Accept-Language": "en-US, en;q=0.5; charset=UTF-8"}
i=0

start = 1
end = 0

mainCounter = 1

#IMDB URL to be accessed
parentUrl  = 'https://www.imdb.com'
childUrl = '/search/title?year=2017&title_type=feature'

#Loop created to scrape multiple pages
i=0
for i in range(10):

    if mainCounter % 200 == 0:
        #Using Pandas Framework
        movie_ratings = pd.DataFrame({
                            'Movie': names,
                            'Genre': genre,
                            'Runtime': runtime,
                            'Imdb_Rating': imdb_ratings,
                            'Metascore_Rating': metascores,
                            })

        print(movie_ratings.info())

        movie_ratings.to_csv('batch-10-' + str(mainCounter) +'.csv', index=False )
        names = []
        imdb_ratings = []
        metascores = []
        genre = []
        runtime = []


    mainUrl = parentUrl + childUrl
    response = get(mainUrl) #Fetches info from the URL 

    page_html = BeautifulSoup(response.text, 'html.parser') #BeautifulSoup converts it into a soup object

    mv_containers = page_html.find_all('div', class_ = 'lister-item mode-advanced')

    next = str(page_html.find('a',class_= 'lister-page-next next-page'))
    startIndex = next.find('href="') + 6
    endIndex = next.find('">Next')

    childUrl = next[startIndex:endIndex]

    for container in mv_containers:

        name = container.h3.a.text
        names.append(str(name).strip()) #Fetching name of the movie

        if container.strong is not None:
            imdb = float(container.strong.text)
        else:
            imdb = 0.0
        imdb_ratings.append(imdb) #Fetching IMDB Ratings 

        if container.find('div', class_ = 'ratings-metascore') is not None:
            m_score = container.find('span', class_ = 'metascore').text
        else:
            m_score = 0.0
        metascores.append(int(m_score)) #Fetching Metascores Rating

        if container.find('span', class_ = 'genre') is not None:
            genreData = str(container.find('span', class_ = 'genre').text)
        else:
            genreData = 'NA'
        genre.append(str(genreData).strip()) #Fetching Genre information

        if container.find('span', class_ = 'runtime') is not None:
            run = str(container.find('span', class_ = 'runtime').text)
        else:
            run = 'NA'
        runtime.append(str(run).strip()) #Fetching runtime of the movie

    mainCounter = mainCounter + 1
    i=i+1

#Storing the values in a data frame
movie_ratings = pd.DataFrame({
                            'Movie': names,
                            'Genre': genre,
                            'Runtime': runtime,
                            'Imdb_Rating': imdb_ratings,
                            'Metascore_Rating': metascores
                            })

#Exporting the data frame contents to a CSV file 
movie_ratings.to_csv('Films_list_1.csv', index=False )

In [2]:
Result_1 = pd.read_csv('Films_list_1.csv') #Read CSV file
Result_1.head() #Display the first few records from the CSV file

,Movie,Genre,Runtime,Imdb_Rating,Metascore_Rating
0,The Upside,"Comedy, Drama",126 min,6.2,45
1,The Greatest Showman,"Biography, Drama, Musical",105 min,7.6,48
2,The Wife,Drama,99 min,7.3,77
3,Get Out,"Horror, Mystery, Thriller",104 min,7.7,84
4,Spider-Man: Homecoming,"Action, Adventure, Sci-Fi",133 min,7.5,73


In [3]:
import requests # to make TMDB API calls
import locale 
locale.setlocale( locale.LC_ALL, '' )
from time import sleep
from random import randint
import pandas as pd
import csv

api_key = "21aefd57526da8bc838461554ca6cc49" #API KEY used to access the TMDB API 

pages = [str(i) for i in range(1,20)] 

columns = ['Movie','Release_date','Popularity']

#Looping to fetch information from multiple pages 
for page in pages:
    response = requests.get('https://api.themoviedb.org/3/discover/movie?api_key=' +  api_key + '&primary_release_year=2017' + '&page=' + page)

    movies = response.json() #Storing the JSON contents retrieved 

    movies_list = movies['results'] 

    df = pd.DataFrame(columns=columns)
    
    #Saving the title, release_date and popularity info in a data frame
    for film in movies_list:
        df.loc[len(df)]=[film['title'],film["release_date"],film["popularity"]] 
    
    #Saving the dataframe contents to a CSV file
    with open('Films_list_2.csv', 'a') as f:
        export_csv = df.to_csv(f, index = None, header=True)

In [4]:
Result_2 = pd.read_csv('Films_list_2.csv') #Reading the CSV file
Result_2.head()  #Displaying the first few rows 

,Movie,Release_date,Popularity
0,The Ash Lad: In the Hall of the Mountain King,2017-09-29,65.911
1,Thor: Ragnarok,2017-10-25,46.242
2,Justice League,2017-11-15,40.615
3,Star Wars: The Last Jedi,2017-12-13,37.97
4,Spider-Man: Homecoming,2017-07-05,35.315


In [5]:
Movies_CSV = pd.read_csv('Films_List_3.csv') #Read CSV file

In [6]:
Movies_CSV.head() #Displaying the first few rows 

,Movie,Votes,Director,Stars,Description
0,The Upside,8424,Neil Burger,"Kevin Hart,Bryan Cranston,Nicole Kidman,Aja Na...",A comedic look at the relationship between a w...
1,The Greatest Showman,186173,Michael Gracey,"Hugh Jackman,Michelle Williams,Zac Efron,Zendaya",Celebrates the birth of show business and tell...
2,The Wife,10244,Björn Runge,"Glenn Close,Jonathan Pryce,Max Irons,Christian...",A wife questions her life choices as she trave...
3,Get Out,363414,Jordan Peele,"Daniel Kaluuya,Allison Williams,Bradley Whitfo...",A young African-American visits his white girl...
4,Spider-Man: Homecoming,387924,Jon Watts,"Tom Holland,Michael Keaton,Robert Downey Jr.,M...",Peter Parker balances his life as an ordinary ...


In [7]:
import pandas as pd
Movies_scrapped = pd.read_csv('Films_list_1.csv') #Read CSV file
Movies_API = pd.read_csv('Films_list_2.csv')
Movies_scrapped = Movies_scrapped.set_index('Movie') #Set index
Movies_API = Movies_API.set_index('Movie')
Movie_list_join_1 = Movies_scrapped.join(Movies_API, on=['Movie'], rsuffix='_Release_date') #Joining two data frames based on Movie value
Movies_CSV = pd.read_csv('Films_list_3.csv')
Movies_CSV = Movies_CSV.set_index('Movie')
Movie_final_join = Movie_list_join_1.join(Movies_CSV, on=['Movie'], rsuffix='_Votes') #Joining two data frames based on Movie value
Movies_final_list = Movie_final_join.dropna(how='any',axis=0) #Eliminating rows containing null values
Movies_final_list.to_csv('Movies_Database.csv', index=True) #Writing the data frame to a CSV file

In [8]:
Final_Result = pd.read_csv('Movies_Database.csv') #Read CSV file 
Final_Result.head() #Display the first few rows 

,Movie,Genre,Runtime,Imdb_Rating,Metascore_Rating,Release_date,Popularity,Votes,Director,Stars,Description
0,The Greatest Showman,"Biography, Drama, Musical",105 min,7.6,48,2017-12-20,28.796,186173,Michael Gracey,"Hugh Jackman,Michelle Williams,Zac Efron,Zendaya",Celebrates the birth of show business and tell...
1,Get Out,"Horror, Mystery, Thriller",104 min,7.7,84,2017-02-24,25.046,363414,Jordan Peele,"Daniel Kaluuya,Allison Williams,Bradley Whitfo...",A young African-American visits his white girl...
2,Spider-Man: Homecoming,"Action, Adventure, Sci-Fi",133 min,7.5,73,2017-07-05,35.315,387924,Jon Watts,"Tom Holland,Michael Keaton,Robert Downey Jr.,M...",Peter Parker balances his life as an ordinary ...
3,Thor: Ragnarok,"Action, Adventure, Comedy",130 min,7.9,74,2017-10-25,46.242,427652,Taika Waititi,"Chris Hemsworth,Tom Hiddleston,Cate Blanchett,...","Thor is imprisoned on the planet Sakaar, and m..."
4,"Three Billboards Outside Ebbing, Missouri","Comedy, Crime, Drama",115 min,8.2,88,2017-11-10,21.367,317218,Martin McDonagh,"Frances McDormand,Woody Harrelson,Sam Rockwell...",A mother personally challenges the local autho...
